In diesem Notebook wollen wir verschiedene Funktionen für Datenbanken erstellen und sammeln.

In [1]:
%whos

Interactive namespace is empty.


In [2]:
### Importieren der notwendigen Module für die Kommunikation mit einer Datenbank
import mysql.connector 
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import exc

In [3]:
### Um mit einer Datenbank zu kommunizieren benötigen wir einen Connection String
#Definieren des Connection-Strings
str_db_typ = 'mysql+mysqlconnector://' #Treiberinformationen
str_db_user = 'root'                   #Benutzer, der auf die Datenbank zugreift
str_db_password = ''                   #Passwort des Benutzers, der auf die Datenbank zugreift. 
str_db_adr = '@localhost'              #Adresse der Datenbank im Netzwerk
str_db_schema = '/northwind'           #Der Name der Datenbank, mit der wir standardmäßig arbeiten wollen.

str_connection = str_db_typ + str_db_user + str_db_password + str_db_adr + str_db_schema

### Ab hier sammeln wir unsere Funktionen

In [4]:
### Kann man nicht eine Funktion schreiben, die immer einen Connection String auswirft?
def create_constr(db_typ='mysql+mysqlconnector://', db_user='root',
                  db_password='', db_adr='@localhost/', db_schema=''):
    '''Diese Funktion erzeugt uns einen Connection String.'''
    try: 
        str_con = f'{db_typ}{db_user}{db_password}{db_adr}{db_schema}'
        print(f'Folgende Verbindung wurde erzeugt: {str_con}')
        return str_con
    except:
        print('Irgendetwas lief schwief.')
        return

In [5]:
create_constr(db_schema='northwind')

Folgende Verbindung wurde erzeugt: mysql+mysqlconnector://root@localhost/northwind


'mysql+mysqlconnector://root@localhost/northwind'

### Wie können wir eine Verbindung zur Datenbank aufbauen?

In [6]:
create_engine(create_constr(db_schema='northwind'))

Folgende Verbindung wurde erzeugt: mysql+mysqlconnector://root@localhost/northwind


Engine(mysql+mysqlconnector://root@localhost/northwind)

In [7]:
engine_0 = create_engine(create_constr(db_schema='northwind'))

Folgende Verbindung wurde erzeugt: mysql+mysqlconnector://root@localhost/northwind


### Was für Datenbanken sind jetzt gerade eigentlich da?

In [8]:
pd.read_sql('SHOW DATABASES', engine_0)

,Database
0,cult_trd_prd
1,information_schema
2,mysql
3,northwind
4,performance_schema
5,phpmyadmin
6,test123
7,uebung


### Wie erzeuge ich eine neue Datenbank?

In [9]:
pd.read_sql('CREATE DATABASE IF NOT EXISTS test567', engine_0)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [10]:
from sqlalchemy import exc

In [11]:
dir(exc)

['AmbiguousForeignKeysError',
 'ArgumentError',
 'CircularDependencyError',
 'CompileError',
 'DBAPIError',
 'DataError',
 'DatabaseError',
 'DisconnectionError',
 'DontWrapMixin',
 'IdentifierError',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'InvalidRequestError',
 'InvalidatePoolError',
 'NoForeignKeysError',
 'NoInspectionAvailable',
 'NoReferenceError',
 'NoReferencedColumnError',
 'NoReferencedTableError',
 'NoSuchColumnError',
 'NoSuchModuleError',
 'NoSuchTableError',
 'NotSupportedError',
 'ObjectNotExecutableError',
 'OperationalError',
 'ProgrammingError',
 'ResourceClosedError',
 'SADeprecationWarning',
 'SAPendingDeprecationWarning',
 'SAWarning',
 'SQLAlchemyError',
 'StatementError',
 'TimeoutError',
 'UnboundExecutionError',
 'UnmappedColumnError',
 'UnreflectableTableError',
 'UnsupportedCompilationError',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'compat']

In [12]:
def exec_sql(str_sql, engine):
    '''Diese Funktion soll jedes SQL-Statement ausführen können, egal ob etwas 
    zurückgeliefert wird oder nicht.'''
    try:
        data = pd.read_sql(str_sql, engine)
        print('Es wurden Daten abgerufen.')
        return data
    except exc.ResourceClosedError as rce:
        print('Der Befehl wurde ausgeführt. Es wurden keine Daten zurückgeliefert.')
        print()
        print(rce)
        return pd.DataFrame()
    except exc.ProgrammingError as per:
        print('Es ist ein Fehler im SQL-Statement.')
        print()
        print(per)
        return 
    except exc.DatabaseError as dbe:
        print(dbe)
        return     
    except:
        print('Es ist ein unbekannter Fehler aufgetreten. Schauen sie in das Error-Log.')
        raise    

In [13]:
exec_sql('CREATE DATABASE IF NOT EXISTS test567', engine_0)

Der Befehl wurde ausgeführt. Es wurden keine Daten zurückgeliefert.

This result object does not return rows. It has been closed automatically.


""


In [14]:
exec_sql('DROP DATABASE test567', engine_0)

Der Befehl wurde ausgeführt. Es wurden keine Daten zurückgeliefert.

This result object does not return rows. It has been closed automatically.


""


### Wie kann man eigentlich Zwischenergebnisse sinnvoll historisieren?

In [15]:
pd.Timestamp.now()

Timestamp('2020-04-16 12:07:47.036695')

In [16]:
def add_a_timestamp(data):
    '''Mit dieser Funktion hängen wir jedem DataFrame einen Zeitstempel an.'''
    ### isstance überprüft, ob ein Objekt zu einem beliebigen Typ gehört
    if isinstance(data, pd.DataFrame):
        df = data.copy()
        df['Uploadtime'] =  pd.Timestamp.now()
        return df
    else:
        print('Es wurde kein DataFrame übergeben.')
        return pd.DataFrame()

In [17]:
wine = pd.read_csv('wine.csv')

In [49]:
#add_a_timestamp(wine)

In [18]:
engine_1 = create_engine(create_constr(db_schema='northwind'))

Folgende Verbindung wurde erzeugt: mysql+mysqlconnector://root@localhost/northwind


In [19]:
add_a_timestamp(exec_sql('SELECT * from orders', engine_1))

Es wurden Daten abgerufen.


,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,Uploadtime
0,10248,VINET,5,1994-08-04,1994-09-01,1994-08-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,2020-04-16 12:08:10.462067
1,10249,TOMSP,6,1994-08-05,1994-09-16,1994-08-10,1,11.61,Toms Spezialit├ñten,Luisenstr. 48,M├╝nster,None,44087,Germany,2020-04-16 12:08:10.462067
2,10250,HANAR,4,1994-08-08,1994-09-05,1994-08-12,2,65.83,Hanari Carnes,"Rua do Pa├ºo, 67",Rio de Janeiro,RJ,05454-876,Brazil,2020-04-16 12:08:10.462067
3,10251,VICTE,3,1994-08-08,1994-09-05,1994-08-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France,2020-04-16 12:08:10.462067
4,10252,SUPRD,4,1994-08-09,1994-09-06,1994-08-11,2,51.30,Supr├¬mes d├®lices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium,2020-04-16 12:08:10.462067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1996-06-04,1996-07-02,NaT,2,24.95,Pericles Comidas cl├ísicas,Calle Dr. Jorge Cash 321,M├®xico D.F.,None,05033,Mexico,2020-04-16 12:08:10.462067
826,11074,SIMOB,7,1996-06-05,1996-07-03,NaT,2,18.44,Simons bistro,Vinb├ªltet 34,K├©benhavn,None,1734,Denmark,2020-04-16 12:08:10.462067
827,11075,RICSU,8,1996-06-05,1996-07-03,NaT,2,6.19,Richter Supermarkt,Starenweg 5,Gen├¿ve,None,1204,Switzerland,2020-04-16 12:08:10.462067
828,11076,BONAP,4,1996-06-05,1996-07-03,NaT,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France,2020-04-16 12:08:10.462067


### Aber, aber, aber... wie würde man das denn auf einer Datenbank anwenden?

Eine Gute Beschreibung der Vor- und Nachteile von Datenbank-Architektur-Konzepten findet sich hier:

https://medium.com/photobox-technology-product-and-design/practical-tips-to-get-the-best-out-of-data-building-tool-dbt-part-1-8cfa21ef97c5

<img src='database_structures.png'>

In [24]:
engine_0 = create_engine(create_constr())

Folgende Verbindung wurde erzeugt: mysql+mysqlconnector://root@localhost/


In [33]:
exec_sql('show databases',engine_0)

Es wurden Daten abgerufen.


,Database
0,cdl
1,cult_trd_prd
2,information_schema
3,mysql
4,northwind
5,performance_schema
6,phpmyadmin
7,test
8,test123
9,uebung


### Anlegen eines Schemas (Datenbank/Namensraum)

In [31]:
exec_sql('create database if not exists CDL',engine_0)

Der Befehl wurde ausgeführt. Es wurden keine Daten zurückgeliefert.

This result object does not return rows. It has been closed automatically.


""


In [30]:
wine.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [32]:
add_a_timestamp(wine)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline,Uploadtime
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,2020-04-16 15:16:56.728707
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,2020-04-16 15:16:56.728707
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,2020-04-16 15:16:56.728707
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,2020-04-16 15:16:56.728707
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,2020-04-16 15:16:56.728707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2020-04-16 15:16:56.728707
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,2020-04-16 15:16:56.728707
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,2020-04-16 15:16:56.728707
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,2020-04-16 15:16:56.728707


In [35]:
engine_2 = create_engine(create_constr(db_schema='CDL'))

Folgende Verbindung wurde erzeugt: mysql+mysqlconnector://root@localhost/CDL


In [49]:
df = pd.read_csv('wine.csv')

In [50]:
add_a_timestamp(df).to_sql(name='wine', con=engine_2,if_exists='append', index=False, chunksize=50)

In [47]:
exec_sql('select * from wine',engine_2)

Es wurden Daten abgerufen.


,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline,Uploadtime
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,2020-04-16 15:23:29
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,2020-04-16 15:23:29
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,2020-04-16 15:23:29
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,2020-04-16 15:23:29
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,2020-04-16 15:23:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2020-04-16 15:24:33
1064,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,2020-04-16 15:24:33
1065,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,2020-04-16 15:24:33
1066,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,2020-04-16 15:24:33


In [54]:
wine.head(1)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065


In [55]:
df.head(1)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135


In [52]:
df = pd.read_csv('cars.csv')

In [56]:
add_a_timestamp(df).to_sql(name='wine', con=engine_2,if_exists='append', index=False, chunksize=50)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'Make' in 'field list'
[SQL: INSERT INTO wine (`Make`, `Model`, `Year`, `Engine Fuel Type`, `Engine HP`, `Engine Cylinders`, `Transmission Type`, `Driven_Wheels`, `Number of Doors`, `Market Category`, `Vehicle Size`, `Vehicle Style`, `highway MPG`, `city mpg`, `Popularity`, `MSRP`, `Uploadtime`) VALUES (%(Make)s, %(Model)s, %(Year)s, %(Engine Fuel Type)s, %(Engine HP)s, %(Engine Cylinders)s, %(Transmission Type)s, %(Driven_Wheels)s, %(Number of Doors)s, %(Market Category)s, %(Vehicle Size)s, %(Vehicle Style)s, %(highway MPG)s, %(city mpg)s, %(Popularity)s, %(MSRP)s, %(Uploadtime)s)]
[parameters: ({'Make': 'BMW', 'Model': '1 Series M', 'Year': 2011, 'Engine Fuel Type': 'premium unleaded (required)', 'Engine HP': 335.0, 'Engine Cylinders': 6.0, 'Transmission Type': 'MANUAL', 'Driven_Wheels': 'rear wheel drive', 'Number of Doors': 2.0, 'Market Category': 'Factory Tuner,Luxury,High-Performance', 'Vehic

In [57]:
def upload_dataframe(df, tablename, con):
    '''Mit dieser Funktion wollen wir pd.to_sql so erweitern, dass wir Daten
    historisieren...'''
    try:
        data = df.copy()
        data = add_a_timestamp(data)
        data.to_sql(name=tablename, con=con,if_exists='append', index=False, chunksize=50)
        print('Datensatz wurde geladen.')
        sql = f'SELECT COUNT(DISTINCT Uploadtime) FROM {tablename}'
        i_version = exec_sql(sql, con).iloc[0,0]
        print(f'Der Datensatz wurde in der Version {i_version} geladen.')
        return i_version
    except exc.ProgrammingError as pre:
        print('Der Dataframe passt nicht zur Struktur der vorhandenen Tabelle')
        print()
        print(pre)
        return
    except:
        print('Es ist ein unbekannter Fehler aufgetaucht.')
        raise

In [58]:
exec_sql('select * from wine',engine_2)

Es wurden Daten abgerufen.


,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline,Uploadtime
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,2020-04-16 15:23:29
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,2020-04-16 15:23:29
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,2020-04-16 15:23:29
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,2020-04-16 15:23:29
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,2020-04-16 15:23:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2020-04-16 15:25:58
1242,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,2020-04-16 15:25:58
1243,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,2020-04-16 15:25:58
1244,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,2020-04-16 15:25:58


In [60]:
df =  pd.read_csv('wine.csv')

In [61]:
upload_dataframe(df,'wine', engine_2)

Datensatz wurde geladen.
Es wurden Daten abgerufen.
Der Datensatz wurde in der Version 8 geladen.


8

In [62]:
df = pd.read_csv('cars.csv')

In [63]:
upload_dataframe(df,'wine', engine_2)

Der Dataframe passt nicht zur Struktur der vorhandenen Tabelle

(mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'Make' in 'field list'
[SQL: INSERT INTO wine (`Make`, `Model`, `Year`, `Engine Fuel Type`, `Engine HP`, `Engine Cylinders`, `Transmission Type`, `Driven_Wheels`, `Number of Doors`, `Market Category`, `Vehicle Size`, `Vehicle Style`, `highway MPG`, `city mpg`, `Popularity`, `MSRP`, `Uploadtime`) VALUES (%(Make)s, %(Model)s, %(Year)s, %(Engine Fuel Type)s, %(Engine HP)s, %(Engine Cylinders)s, %(Transmission Type)s, %(Driven_Wheels)s, %(Number of Doors)s, %(Market Category)s, %(Vehicle Size)s, %(Vehicle Style)s, %(highway MPG)s, %(city mpg)s, %(Popularity)s, %(MSRP)s, %(Uploadtime)s)]
[parameters: ({'Make': 'BMW', 'Model': '1 Series M', 'Year': 2011, 'Engine Fuel Type': 'premium unleaded (required)', 'Engine HP': 335.0, 'Engine Cylinders': 6.0, 'Transmission Type': 'MANUAL', 'Driven_Wheels': 'rear wheel drive', 'Number of Doors': 2.0, 'Market Category': '

### Wie geht denn nochmal... ?

In [64]:
#... wie können wir eine Tabelle leeren?
exec_sql('TRUNCATE TABLE wine', engine_2)

Der Befehl wurde ausgeführt. Es wurden keine Daten zurückgeliefert.

This result object does not return rows. It has been closed automatically.


""


In [65]:
#... hat das geklappt?
exec_sql('SELECT * FROM wine', engine_2)

Es wurden Daten abgerufen.


,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline,Uploadtime


In [67]:
#... wie können wir denn die Struktur einer Tabelle anzeigen
exec_sql('SHOW COLUMNS FROM wine', engine_2)

Es wurden Daten abgerufen.


,Field,Type,Null,Key,Default,Extra
0,Class,bigint(20),YES,,None,
1,Alcohol,double,YES,,None,
2,Malic acid,double,YES,,None,
3,Ash,double,YES,,None,
4,Alcalinity of ash,double,YES,,None,
5,Magnesium,bigint(20),YES,,None,
6,Total phenols,double,YES,,None,
7,Flavanoids,double,YES,,None,
8,Nonflavanoid phenols,double,YES,,None,
9,Proanthocyanins,double,YES,,None,


In [68]:
#... wie können wir denn die Struktur einer Tabelle anzeigen (Variante 2)
exec_sql('DESCRIBE wine', engine_2)

Es wurden Daten abgerufen.


,Field,Type,Null,Key,Default,Extra
0,Class,bigint(20),YES,,None,
1,Alcohol,double,YES,,None,
2,Malic acid,double,YES,,None,
3,Ash,double,YES,,None,
4,Alcalinity of ash,double,YES,,None,
5,Magnesium,bigint(20),YES,,None,
6,Total phenols,double,YES,,None,
7,Flavanoids,double,YES,,None,
8,Nonflavanoid phenols,double,YES,,None,
9,Proanthocyanins,double,YES,,None,


In [69]:
engine_3 = create_engine(create_constr(db_schema='northwind'))

Folgende Verbindung wurde erzeugt: mysql+mysqlconnector://root@localhost/northwind


In [70]:
exec_sql('show tables', engine_3)

Es wurden Daten abgerufen.


,Tables_in_northwind
0,categories
1,customers
2,employees
3,order details
4,orders
5,products
6,shippers
7,suppliers


In [72]:
exec_sql('select * from customers', engine_3).head(1)

Es wurden Daten abgerufen.


,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,Image,ImageThumbnail
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,None,None


In [81]:
exec_sql('show columns from customers', engine_3)

Es wurden Daten abgerufen.


,Field,Type,Null,Key,Default,Extra
0,CustomerID,varchar(5),NO,PRI,None,
1,CustomerName,varchar(40),YES,MUL,None,
2,ContactName,varchar(30),YES,,None,
3,ContactTitle,varchar(30),YES,,None,
4,Address,varchar(60),YES,,None,
5,City,varchar(15),YES,MUL,None,
6,Region,varchar(15),YES,MUL,None,
7,PostalCode,varchar(10),YES,MUL,None,
8,Country,varchar(15),YES,,None,
9,Phone,varchar(24),YES,,None,


In [80]:
### Wie können wir bspw. eine Spalte in einer Tabelle umbenennen
exec_sql('ALTER TABLE CUSTOMERS CHANGE CompanyName CustomerName varchar(40)', engine_3)

Der Befehl wurde ausgeführt. Es wurden keine Daten zurückgeliefert.

This result object does not return rows. It has been closed automatically.


""


In [82]:
###Und wie können wir eine ganze Tabelle umbenennen?
exec_sql('RENAME TABLE `order details` TO order_details', engine_3)

Der Befehl wurde ausgeführt. Es wurden keine Daten zurückgeliefert.

This result object does not return rows. It has been closed automatically.


""
